In [ ]:
import requests
import json

APPID = "あなたの Client ID（アプリケーション ID）"
URL = "https://jlp.yahooapis.jp/NERService/V1/extract"

def get_entities(text):
    # APIリクエストのペイロード
    payload = {
        "jsonrpc": "2.0",
        "id": "1",
        "method": "jlp.nerservice.extract",
        "params": {
            "q": text
        }
    }

    # APIリクエストのヘッダー
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Yahoo AppID: {}".format(APPID),
    }

    # APIリクエストの送信
    response = requests.post(URL, json=payload, headers=headers)

    # レスポンスの解析
    if response.status_code == 200:
        try:
            data = response.json()
            entities = data.get("result", {}).get("entityList", [])
            return entities
        except json.JSONDecodeError as e:
            print("JSONDecodeError:", e.msg)
            return []
    else:
        print("Error: Invalid response")
        return []

In [ ]:
def mask_entities(text, entities):
    masked_text = text
    entity_counters = {}
    entity_map = {}

    for entity in entities:
        entity_type = entity["type"]
        if entity_type not in entity_counters:
            entity_counters[entity_type] = 1
        else:
            entity_counters[entity_type] += 1

        tag = f"<{entity_type}_{entity_counters[entity_type]}>"
        entity_map[tag] = entity["text"]
        masked_text = masked_text.replace(entity["text"], tag)

    return masked_text, entity_map

In [ ]:
def unmask_entities(masked_text, entity_map):
    unmasked_text = masked_text
    for tag, original_text in entity_map.items():
        unmasked_text = unmasked_text.replace(tag, original_text)
    return unmasked_text

In [ ]:
sample_text = """
2023年12月7日、東京本社の会議室Aにて、プロジェクト「Phoenix」の進捗状況に関するミーティングが行われました。参加者は、プロジェクトマネージャーの田中太郎さん、開発リーダーの山田花子さん、デザインチームの佐藤次郎さん、マーケティング部の鈴木一郎さん、そして外部コンサルタントのジョン・スミスさんです。
まず、田中太郎さんがプロジェクト全体の進捗状況を報告しました。現在、プロジェクトは予定通り進行しており、主要なマイルストーンはすべて達成されています。次に、山田花子さんが開発チームの進捗状況を報告しました。新しい機能の実装は順調に進んでおり、バグ修正もほぼ完了しています。
続いて、佐藤次郎さんがデザインの進捗状況を報告しました。ユーザーインターフェースのデザインはほぼ完成しており、ユーザビリティテストも順調に進んでいます。鈴木一郎さんは、マーケティング戦略について報告しました。新しい広告キャンペーンが開始され、初日の売上は100万円を超えました。また、顧客満足度調査の結果、90％の顧客が新しい製品「Phoenix」に満足していることがわかりました。
ジョン・スミスさんは、外部からの視点でプロジェクトの評価を行いました。彼は、プロジェクトの進捗状況に満足しており、特に開発チームの効率性を高く評価しました。また、彼は今後のリスク管理についても提案を行い、特にサイバーセキュリティ対策の強化を推奨しました。
最後に、田中太郎さんが今後のスケジュールについて説明しました。次回のミーティングは2023年12月21日に予定されており、場所は同じく東京本社の会議室Aです。ミーティングは10時から開始される予定です。参加者全員がスケジュールを確認し、ミーティングは終了しました。
"""


# 固有表現を抽出
entities = get_entities(sample_text)

# 固有表現をマスキング
masked_text, entity_map = mask_entities(sample_text, entities)

# マスクされたテキストを表示
print("Masked Text:", masked_text)

In [ ]:
summary_text = """
プロジェクト「<ARTIFACT_1>」進捗状況ミーティング概要
日時: <DATE_1>
場所: <LOCATION_1>本社 会議室A
参加者:

プロジェクトマネージャー: <PERSON_1>
開発リーダー: <PERSON_2>
デザインチーム: <PERSON_3>
マーケティング部: <PERSON_4>
外部コンサルタント: <PERSON_5>
報告内容
プロジェクト全体の進捗（<PERSON_1>）

予定通り進行中
主要なマイルストーン達成
開発チームの進捗（<PERSON_2>）

新機能の実装順調
バグ修正ほぼ完了
デザインの進捗（<PERSON_3>）

ユーザーインターフェースのデザインほぼ完成
ユーザビリティテスト順調
マーケティング戦略（<PERSON_4>）

新広告キャンペーン開始
<DATE_2>の売上が<MONEY_1>を超える
顧客満足度調査で<PERCENT_1>%の顧客が満足
外部評価（<PERSON_5>）

プロジェクト進捗に満足
開発チームの効率性を高評価
サイバーセキュリティ対策の強化を提案
今後のスケジュール
次回ミーティング: <DATE_3>
場所: <LOCATION_1>本社 会議室A
時間: <TIME_1>開始
"""

# 要約文を復元
unmasked_text = unmask_entities(summary_text, entity_map)

# 復元された要約文を表示
print(unmasked_text)